# SYnDiCATE
## SAÉ 2.0 <img src="https://cdn.pixabay.com/photo/2016/11/30/09/27/hacker-1872291_960_720.jpg" width=550 align=right>

**Objectif général**  

Nous sommes une organisation secrète internationale d’activistes "white hackers" qui luttons pour la liberté.  

Nous avons écrit un texte, le MANIFESTO, qui décrit notre organisation, ses idées, ses buts, ses méthodes, ses objectifs. Jusqu’au moment où nous révélerons massivement notre existence au monde,

**le manifesto NE DOIT PAS tomber aux mains de l’ennemi.**  


Pour cette raison, le texte ne sera intégralement disponible sur aucune machine de l’organisation. A la place, il sera fragmenté en plusieurs dizaines de milliers de morceaux épartis aléatoirement sur autant de machines dans le monde.   

Pour gérer ce système nous avons besoin d'un logiciel puissant : le **SYnDICATE** ou SYstème DIstribué de ChAinage TExtuel.

Beaucoup de nos membres sont issus de vos rangs aussi nous proposons aux étudiants du BUT INFO de participer à l'élaboration du syndicate. Effectuez un bon travail et nous considérerons votre recrutement dans l'organisation.

Le but de cette SAE est donc d’étudier des fonctionnalités permettant de manipuler un texte réparti, sans jamais, A AUCUN MOMENT, disposer de l’intégralité du texte.  

Il en va de notre survie.

Il en va de l’avenir du monde.




## Fonctionnement général

Dans cette SAE, on ne manipulera pas le vrai manifesto mais un texte de substitution afin de créer le système. Ce texte a déjà été découpé en tronçons de 1 à 20 mots destinés à être répartis sur des machines. Le fichier *manifesto.txt* contient sur chaque ligne des données à stocker sur une machine. Le fichier *manifesto_short.txt* peut être utile dans la partie 2 si vous travailler sur google colab car vous pouvez manquer de mémoire vive.

In [ ]:

#code pour charger le fichier manifesto.txt depuis google colab
#à l'exécution cliquez sur "choose file" et choisissez le bon fichier.
from google.colab import files
import io
uploaded = files.upload()
lignes = uploaded["manifesto.txt"].decode("utf-8").splitlines()

On extrait un tableau qui contient les lignes du texte, et on affiche les 20 premières pour que vous voyez à quoi ça ressemble.

In [ ]:

print("nombre de lignes :", len(lignes) )
print("20 premières lignes:\n")
for l in lignes[:20]:
    print(l)


Ce fichier de presque 10000 lignes (pour la version longue) contient sur sa première ligne l'adresse IP fictive d'une machine appelée FIRST. C'est l'adresse IP de la machine d'accès au réseau secret syndicate.

Ensuite, chaque ligne à partir de la troisième contient : 
- l'adresse IP d'une machine
- sa cargaison (un morceau de phrase) = l'information stockée sur cette machine
- l'adresse IP de la machine suivante

# Partie 0 : Fonctionnement centralisé (non sécurisé) -- algorithmes itératifs

Il va de soi que l'existence même de ce fichier *manifesto.txt* est problématique du point de vue de notre objectif et qu'il ne sert ici qu'à concevoir le système sur lequel nous allons. À terme, la seule façon d'obtenir des informations sur le texte sera de procéder de façon sécurisée, de façon **chaînée**.

Néanmoins, pour se familiariser avec le fonctionnement, dans cette partie nous allons créer un annuaire des machines qui peut accéder directement à toutes les machines.

Mais tout d'abord, nous créons une classe pour représenter les machines.

In [ ]:
class Machine:
    """
    permet de stocker les informations d'une machine
    """
    def __init__(self, ip, cargo, next_ip):
        self.ip = ip              #str: ip de cette machine
        self.cargo = cargo        #str: informations stockées
        self.next_ip = next_ip    #str : ip de la machine suivante

    
    def executer(self, fx, arguments):
        """
        utilisé à partir de la partie 2 -- ignorer pour l'instant
        """
        return fx(self, *arguments)


Exemple d'un objet de la classe Machine, et on affiche ses informations ensuite.

In [ ]:
m1 = Machine("103.345.232.345", "salut je suis un exemple de machine", "345.765.234.654")
print(m1.ip)
print(m1.cargo)
print(m1.next_ip)

On vous donne maintenant une classe permettant de créer un objet qui va lire le fichier de départ et le parser, créer toutes les machines correspondantes et simuler le réseau mondial.

In [ ]:
class IP_table:
    """permet de stocker l'ensemble des machines indexées par IP"""

    def __init__(self, tab_lignes):
        """
        crée la IP_table à partir du tableau des lignes décrivant les machines
        """
        self.table = self.charger_machines(tab_lignes)


    def charger_machines(self, tab_lignes):
        """
        crée un dictionnaire de machines à partir du tableau des lignes décrivant les machines
        """
        n = len(tab_lignes)
        first = tab_lignes[0].split('#')[1].strip() #on découpe la première ligne et on récupère l'adresse first
        table = {}

        for l in lignes[2:]:
            ip, cargo, ip_next = l.split('#')[:3] 
            table[ip] = Machine(ip, cargo, ip_next)  #on découpe la ligne

        table['FIRST'] = table[first]
        del table[first]
        return table

    def get_machine(self, ip):
        """renvoie la machine qui a une ip donnée"""
        if ip in self.table:
            return self.table[ip]
 

Vue de l'extérieur, on a juste à créer la classe, et ensuite on appelle sa méthode *get_machine*. On est pas obligés de comprendre les détails internes.

In [ ]:
ip_table = IP_table(lignes)
f = ip_table.get_machine('FIRST')
print(f.cargo)

Il semble que la première machine ne contienne que le mot "PART". Récupérons l'adresse ip de la machine suivante et affichons, et recommençons sur quelques machines.

In [ ]:
print(f.cargo, end = ' ')
m1 = ip_table.get_machine(f.next_ip)
print(m1.cargo, end = ' ')
m2 = ip_table.get_machine(m1.next_ip)
print(m2.cargo, end = ' ')

Pas mal, on a réussi à affichier la première phrase. Mais c'est quand même un peu laborieux. Ha mais au fait au lieu de répéter du code on peut faire une boucle non ?

**OBOULO** Effectuez une boucle afin d'afficher le contenu des 100 premières machines.

In [ ]:
#soluce
ip =  'FIRST'
for _ in range(100):
    m = ip_table.get_machine(ip)
    print(m.cargo, end = ' ')
    ip = m.next_ip


Maintenant, affichez carrément le texte complet. La dernière machine de la chaîne a un champ *ip_next* qui vaut "LAST", ce qui permet de s'arrêter.

In [ ]:
#soluce
ip =  'FIRST'
while ip != 'LAST':
    m = ip_table.get_machine(ip)
    print(m.cargo, end = ' ')
    ip = m.next_ip

En fait, tout ne s'affiche pas ici car le texte est trop long.

Utilisez le code précédent afin de compléter la fonction suivante. Il faut ajouter une espace entre les morceaux de texte pour avoir une chaîne propre.

In [ ]:
def str_texte_complet(ip_table):
    """renvoie une str qui contient le texte complet (pas de print)"""
    pass


Pour vérifier : en ajoutant une espace après chaque morceau de texte, la longueur totale de la chaîne obtenue est de 585273.

In [ ]:
#soluce
def str_texte_complet(ip_table):
    """renvoie une str qui contient le texte complet  (pas de print)"""
    s = ""
    ip =  'FIRST'
    while ip != 'LAST':
        m = ip_table.get_machine(ip)
        s += m.cargo + " "
        ip = m.next_ip
    return s
print(len(str_texte_complet(ip_table)))

**exo** Afin de tester notre travail, réalisez une fonction qui renvoie la liste des cargaisons qui contiennent un mot donné. Nous vous laissons trouver comment utiliser une instruction python simple pour savoir si une str contient un mot. Bien entendu, dans la suite du travail cette fonction est utilisée uniquement pour tester, lorsque le système sera déployé elle sera interdite.

Pour vérifier : il y a 25 machines dont la cargaison contient le mot 'cold'.

In [ ]:
#soluce
def cargo_avec_mot(ip_table, mot):
    """renvoie une str qui contient le texte complet  (pas de print)"""
    l = []
    ip =  'FIRST'
    while ip != 'LAST':
        m = ip_table.get_machine(ip)
        if mot in m.cargo:
            l.append(m.cargo)
        ip = m.next_ip
    return l
res = cargo_avec_mot(ip_table, "cold")
print(res)
print(len(res))

**NB** Dans le but d'étudier coment le réseau secret va être réaliser, on s'interdit de réaliser une structure extérieure comme un tableau qui contiendrait la totalité des machines dans l'ordre, et on s'oblige à passer de machine en machine, toujours par ip_table. On va aussi **s'interdire de reconstituer le texte entier**.

**exo** Ecrivez maintenant une fonction qui compte le nombre total de lettres dans le texte complet, mais sans le reconstituer. Indication : pour chaque machine, il faut maintenant compter directement la longueur de la cargaison,
plus 1 pour l'espace.

In [ ]:
def longueur_texte(ip_table):
    pass

**exo** idem mais maintenant on compte les mots. Les méthodes standards de la classe **str** peuvent être utiles.

**exo** écrivez maintenant une fonction qui renvoie le plus long mot du texte entier, toujours en procédant de machine en machine et sans reconstituer le texte entier ! En cas d'égalité de longueur, renvoyer le premier.

**exo**  écrivez une fonction qui remplace chaque occurence d'un mot dans le texte entier, par un autre mot donné. Par exemple, remplacez toutes les occurences de "cold" par "hot", ça réchauffe.
NB : pour faire cela vous pouvez modifier directement m.cargo pour chaque machine m

**exo**  A la fin du texte, on voudrait ajouter quelques phrases.
Ecrivez une fonction qui va ajouter une nouvelle machine, avec une IP donnée, à la fin de la liste chaînée des machines. Pour créer la machine, procédez comme expliqué au début de ce document, et mettez à jour ip_table.table qui est un dictionnaire. Pour vérifiez, réutilisez les fonctions précédentes sur ce texte mis à jour.

**exo** Ecrivez maintenant une fonction semblable, mais qui insère une nouvelle machine avec un texte donné
**juste après** la machine qui contient un mot donné (par exemple, insérer une machine avec le texte "bla bla bla"
avant la première machine dont la cargaison contient le mot "horse"). Testez. Si aucune machine ne contient ce mot,
la fonction ne fait rien.

## Partie 1 : vers un système sécurisé

Dans la partie I, le fonctionnement du système est centralisé, avec des fonctions qui peuvent obtenir directement les informations de chaque machine.
Ceci est à proscrire dans le cadre du système sécurisé que nous voulons construire. Aussi, on va s'interdire, pour une machine m, d'exécuter une fonction sur m qui va directement chercher les informations de la machine m comme m.cargo et m.next_ip

A la place, on va demander à la machine m d'executer une fonction f sur elle-même. La différence est que la personne qui possède la machine m peut examiner le code, l'analyser et décider de l'exécuter ou non, de transmettre les informations ou non. 

Ceci se fait par l'intermédiaire de la fonction suivante.

In [ ]:
def demande_execution(ip_table, ip_machine, fx, arguments):
    """
    demande à la machine d'IP 'ip_machine' d'exécuter la fonction 'fonction'
    sur la liste des arguments donnée, et de renvoyer le résultat.
    fx est une fonction de signature fx(machine, arg1, arg2, ...)
    machine represente la machine sur laquelle la fonction est appelée
    arg1, arg2... correspondent à la liste arguments
    """
    return ip_table.get_machine(ip_machine).executer(fx,arguments)

Pour vous aider à comprendre, voici un exemple de fonction que l'on demande à la machine FIRST d'exécuter sur elle-même.

In [ ]:
def fx_demo(machine, truc1, truc2):
    """la machine va afficher sa cargaison et les chaînes truc1 et truc2
    elle va renvoyer l'ip de la machine suivante"""
    print(machine.cargo)
    print(truc1)
    print(truc2)
    return machine.next_ip

demande_execution(ip_table, 'FIRST', fx_demo, ["mot1", "mot2"])

On ne va plus non plus pouvoir écrire `ipn = m.next_ip`.
A la place, on utilise la fonction suivante :

In [ ]:
def get_ip_next(machine):
    return machine.next_ip
ipn = demande_execution(ip_table, 'FIRST', get_ip_next, [])
print(ipn)

**exo** Refaites quelques fonctions de votre choix (par exemple comptage de lettres, remplacement des mots) dans ce cadre.
A chaque fois, il faut écrire la fonction principale, dont le paramètre est juste ip_table, et la fonction que l'on va demander à chaque machine d'exécuter.

In [ ]:

#soluce
def nb_lettres_machine(machine):
    return len(m.cargo) + 1
def nb_lettres(ip_table):
    nb = 0
    ip =  'FIRST'
    while ip != 'LAST':

        nb += demande_execution(ip_table, ip, nb_lettres_machine,[])
        ip = demande_execution(ip_table, ip, get_ip_next, [])
    return nb
print(nb_lettres(ip_table))


## Partie 2 : Fonctionnement sécurisé en chaîne

**NB** Pour cette partie, il peut être préférable d'utiliser le fichier *manifesto_short.txt*


Du coup, comment faire pour effectuer des traitements sur les machines dans ce protocole ? On donne un exemple ci-dessous pour le comptage des lettres (à voir s'il est légal ou illegal, une machine peut très bien décider de ne pas exéctuter la fonction).

In [ ]:
#code pour charger le fichier manifesto_short.txt depuis google colab
#à l'exécution cliquez sur "choose file" et choisissez le bon fichier.
from google.colab import files
import io
uploaded = files.upload()
lignes = uploaded["manifesto_short.txt"].decode("utf-8").splitlines()

In [ ]:
ip_table = IP_table(lignes)


A partir de maintenant, on va plus loin dans la sécurité. On suppose que chaque
machine ne répondra qu'aux demandes d'éxécution de la machine précédente. Pour cela, chaque machine possèdera un mot de passe permettant de demander à
la machine suivante d'exécuter une fonction, mais elle ne transmettra jamais ce mot de passe à personne.

En pratique, on ne va pas simuler ce mot de passe, mais on va écrire des codes où seulement la machine précédente a le droit de faire une demande d'exécution sur la machine suivante.

Ainsi, personne ne connaît les adresses des machines dans l'ordre. La table `ip_table` connaît les adresses de nombreuses machines (et en réalité d'autres machines n'ayant rien à voir avec notre réseau) mais ne sait pas précisément quelles sont les machines du réseau ni dans quelle ordre elles sont chaînées, à l'exception de la machine FIRST.

In [ ]:
#pour éviter les erreurs sur la profondeur de récursion, on exécute ceci :
import sys
sys.setrecursionlimit(50000)

In [ ]:
N = 0
def total_lettres(machine) :
    """renvoie le nombre total de lettres dans toutes les cargaisons à partir de la machine courante"""
    global N
    N += 1
    if N%1000==0:
        print(N)
        input()
    if machine.next_ip == 'LAST':
        lettres_plus_loin = 0
    else:
        lettres_plus_loin = demande_execution(ip_table, machine.next_ip, total_lettres, [])
    return lettres_plus_loin + 1

def nb_lettres_total_recursif(ip_table):
    return demande_execution(ip_table, 'FIRST', total_lettres, [])
nb_lettres_total_recursif(ip_table)